In [19]:
from u import *
from ut import *
from data import *
from quantized_model import evaluate, get_net
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import distiller

from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy

%load_ext autoreload
%autoreload 2

decoder = (Cache / 'vocab.npy').load()
encoder = get_encoder(decoder)
n_vocab = len(decoder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Quantization Aware Training

In [21]:
c_base = Config(Wiki / 'shallow8_distill_1011_tgt40_agp_t175_lr0.0001', device='cuda', logger=False).load()
c = deepcopy(c_base).var(res=Wiki / 'quant_aware,2').unvar(
    'adaptive_ratio', 'pos_emb', 'hebbian', 'fix_softmax', 'hebbian_T', 'hebbian_gamma'
)
c = c.var(
    model=Proj / 'quantized_model.py',
    compress=c.res / 'distiller.yaml',
    train_batch=1,
    steps_per_epoch=1000,
    step_warmup=0,
    lr=0.0001,
    step_eval=1,
).save(True)
state = c_base.load_state(124000)
print('step', state['step'])
# del state['amp']
del state['opt']
state['step'] = 0
# state['opt']['param_groups'][0]['lr'] = 5e-4
c.save_state(state['step'], state)
(c.res._up / 'quant_aware/distiller.yaml').cp(c.res)

step 124000
Saved model /data/scratch/zxyan/micronet/wikitext-103/quant_aware,2/models/model-0.pth at step 0


In [13]:
net = get_net(c)
net, step = c.init_model(net, step='max', train=False)
data_val = SequentialIterator(c, c.eval_batch, split='valid')
print('Loaded model at step', step)

Loaded model at step 0


In [22]:
print(c.train(env_gpu=0, steps=1))

cd /data/scratch/zxyan/micronet/wikitext-103/quant_aware,2
CUDA_VISIBLE_DEVICES=0 python3 ../../quantized_model.py . steps=1 opt_level=O0


In [ ]:
evaluate(c, data_val, net)

# QAT Processing

In [26]:
c = Config(Wiki / 'quant_aware,2').load()
net = get_net(c)

state = torch.load(c.res / 'models/model-1.pth')
state = state['net']
num_bits = 9
max_int = 2 ** (num_bits - 1) - 1
min_int = - 2 ** (num_bits - 1)

In [27]:
import struct

def float_to_bin(num):
    return bin(struct.unpack('!I', struct.pack('!f', num))[0])[2:].zfill(32)

def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

def get_fp_reduced(x, bits):
    s = float_to_bin(x)
    s = np.array(list(s))
    s[bits:] = '0'
    s = ''.join(s)
    return bin_to_float(s)

In [28]:
new_state = OrderedDict()
for k, p in net.named_modules():
    if k + '.weight_scale' in state:
        weight = state[k + '.weight']
        weight_scale = state[k + '.weight_scale']
        wq = (weight * weight_scale).round()
        assert (wq - weight * weight_scale).abs().max() < 1e-4
        assert ((min_int <= wq) & (wq <= max_int)).all()
        weight_inv_scale = torch.tensor(
            [get_fp_reduced(x, 32 - num_bits) for x in 1 / weight_scale.reshape(-1)]
        ).reshape_as(weight_scale) # mantissa of weight_inv_scale has at most 32 - num_bits
        new_weight = wq * weight_inv_scale # new weight is exactly representable in 32 bits
        new_state[k + '.weight'] = new_weight
    elif k + '.weight' in state:
        new_state[k + '.weight'] = state[k + '.weight']
    if k + '.bias' in state:
        new_state[k + '.bias'] = state[k + '.bias']
    if k + '.fake_q.scale' in state:
        new_state[k + '.max_abs'] = torch.max(state[k + '.fake_q.tracked_max'].abs(), abs(state[k + '.fake_q.tracked_min'].abs()))
        scale = state[k + '.fake_q.scale'] 
        inv_scale = get_fp_reduced(1 / scale, 32 - num_bits) # mantissa of inv_scale has at most 32 - num_bits
        new_state[k + '.inv_scale'] = torch.tensor(inv_scale)
    if k + '.pos_emb' in state:
        new_state[k + '.pos_emb'] = state[k + '.pos_emb']
for k in ['loss.cache_lambda_inv_sigmoid', 'loss.cache_theta_inv_softplus']:
    new_state[k] = state[k]
torch.save(new_state, c.res / 'models/model-1-processed.pth')

# Running QAT

In [ ]:
c = Config(Wiki / 'quant_aware,3', device='cuda:0', distributed=False).load()
import quantized_model
quantized_model.distiller_vs_explicit = 'explicit'
net = get_net(c)

net.load_state_dict(
    torch.load(c.res / 'models/model-1-processed.pth')
)
net = net.to(c.device)

data_val = SequentialIterator(c, c.eval_batch, split='valid')
data_test = SequentialIterator(c, c.eval_batch, split='test')

In [5]:
net.cache_keys = net.cache_values = None
print(evaluate(c, data_val, net))
net.cache_keys = net.cache_values = None
print(evaluate(c, data_test, net))

{'loss': 3.5340301990509033, 'perplexity': 34.26177149866618, 'time': 2.0}
{'loss': 3.5535757541656494, 'perplexity': 34.93802417999064, 'time': 2.0}


# Post Train

In [2]:
quantization = (Wiki / 'quantization').mk()
c_base = Config(Wiki / 'shallow8_cache_ppl33.20_tgt40_grad0_lr0.00010', device='cuda', logger=False).load()
import copy
c = copy.deepcopy(c_base).var(res=quantization).unvar(
    'adaptive_ratio', 'pos_emb', 'hebbian', 'fix_softmax', 'hebbian_T', 'hebbian_gamma'
)
c = c.var(
    model=Proj / 'quantized_model.py',
    compress=c.res / 'distiller.yaml',
    train_batch=9,
    steps_per_epoch=1000,
    step_warmup=0,
    lr=0.0001
).save(True)
state = c_base.load_state('max')
# del state['amp']
del state['opt']
state['step'] = 0
# state['opt']['param_groups'][0]['lr'] = 5e-4
c.save_state(state['step'], state)

Saved model /data/scratch/zxyan/micronet/wikitext-103/quantization/models/model-0.pth at step 0


'/data/scratch/zxyan/micronet/wikitext-103/quantization/models/model-0.pth'

In [3]:
from quantized_model import get_net, evaluate
from data import *
data_val = SequentialIterator(c, c.eval_batch, split='valid')
data_test = SequentialIterator(c, c.eval_batch, split='test')
net = get_net(c)
net, step = c.init_model(net, step='max', train=False)
print('Loaded model at step', step)

Loaded model at step 0


In [4]:
net.loss.cache_keys = net.loss.cache_values = None
print(evaluate(c, data_val, net))
net.loss.cache_keys = net.loss.cache_values = None
print(evaluate(c, data_test, net))

{'loss': 3.5211262702941895, 'perplexity': 33.822500297597784, 'time': 1.0}
{'loss': 3.541309118270874, 'perplexity': 34.51207001194503, 'time': 1.0}


In [6]:
import os
import distiller
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

distiller.utils.assign_layer_fq_names(net)
collector = QuantCalibrationStatsCollector(net)

pretr_stats = c.res / 'pretrained_stats.yaml'
if not pretr_stats.exists():
    with collector_context(collector) as collector:
        evaluate(c, data_val, net)
        collector.save(pretr_stats)

In [19]:
overrides_yaml = """
embed.*:
    fp16: true
projection.*:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(net),
    model_activation_stats=pretr_stats,
    bits_activations=32,
    bits_parameters=32,
    per_channel_wts=True,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED
)

In [22]:
net

Transformer(
  (embed): AdaptiveEmbedding(
    (layers): ModuleList(
      (0): Embedding(3500, 256)
      (1): Embedding(21500, 64)
      (2): Embedding(242735, 4)
    )
    (projections): ModuleList(
      (0): Linear(in_features=64, out_features=256, bias=False)
      (1): Linear(in_features=4, out_features=256, bias=False)
    )
  )
  (dropout1): Dropout(p=0)
  (layers): ModuleList(
    (0): Decoder(
      (ln1): LayerNorm(torch.Size([256]), eps=1e-05, elementwise_affine=True)
      (qkv): Linear(in_features=256, out_features=576, bias=True)
      (out): Linear(in_features=192, out_features=256, bias=False)
      (dropout): Dropout(p=0)
      (ln2): LayerNorm(torch.Size([256]), eps=1e-05, elementwise_affine=True)
      (fc): Sequential(
        (0): Linear(in_features=256, out_features=768, bias=True)
        (1): ReLU(inplace)
        (2): Dropout(p=0)
        (3): Linear(in_features=768, out_features=256, bias=True)
        (4): Dropout(p=0)
      )
    )
    (1): Decoder(
      

In [20]:
stats_before_prepare = deepcopy(quantizer.model_activation_stats)
dummy_input = (torch.zeros(1, 1).long(),) * 2
quantizer.prepare_model(dummy_input)

In [25]:
net.loss.cache_keys = net.loss.cache_values = None
print(evaluate(c, data_val, net.half()))
net.loss.cache_keys = net.loss.cache_values = None
print(evaluate(c, data_test, net.half()))

{'loss': 3.8828125, 'perplexity': 48.560599877464846, 'time': 3.0}
{'loss': 3.90625, 'perplexity': 49.71218131735947, 'time': 4.0}


In [21]:
quantizer.model.loss.cache_keys = quantizer.model.loss.cache_values = None
print(evaluate(c, data_val, quantizer.model))
quantizer.model.loss.cache_keys = quantizer.model.loss.cache_values = None
print(evaluate(c, data_test, quantizer.model))

{'loss': 16.36498260498047, 'perplexity': nan, 'time': 3.0}
{'loss': 16.05936622619629, 'perplexity': nan, 'time': 3.0}
